# Inicialização

In [1]:
# Dependências gerais
import numpy as np

# Dependências locais
from utils.glfw_window import GLFWWindow
from utils.opengl_session import OpenGLSession
from utils.components import OpenGLComponentCompound, Triangles, Rectangle, Square, Circle
from utils.transformations import translate3D, scale3D, zrotate3D, relative_zrotate3D

# Propriedades da janela
WINDOW_WIDTH = 720
WINDOW_HEIGHT = 480
WINDOW_TITLE = "Main"

# Inicialização da janela e da sessão
window = GLFWWindow(WINDOW_WIDTH, WINDOW_HEIGHT, WINDOW_TITLE)
session = OpenGLSession(window, num_dims=2)

# Componentes

In [2]:
def random_color():
    return np.hstack([np.random.normal(0.0, 1.0, size=3), (1.0)])

# Triângulos
triangles = [
    Triangles (
        vertices = np.random.uniform(-0.7, 0.7, size=(3,2)), 
        color = random_color()
    )
    for _ in range(2)
]

# Retângulos e quadrados
rectangles = [
    Square (
        side = 1.8, 
        color = random_color()
    ), 
    Rectangle (
        width = 1.0, 
        height = 1.5, 
        color = random_color()
    )
]

# Círculos
circles = [
    Circle (
        x0=np.random.uniform(-1.0, 1.0), 
        y0=np.random.uniform(-1.0, 1.0), 
        radius=np.random.uniform(0.0, 0.5), 
        color = random_color()
    )
    for _ in range(3)
]

# MIMOSA
mimosa = OpenGLComponentCompound ([

    # Patas
    Square(x0=0.2, y0=0.1, side=0.2, color=random_color()),
    Square(x0=0.6, y0=0.1, side=0.2, color=random_color()),

    # Corpo e cauda
    Rectangle(x0=0.4, y0=0.35, width=0.6, height=0.3, color=random_color()),
    Triangles(vertices=np.array([(0.0, 0.5), (0.1, 0.5), (0.1, 0.6)]), color=random_color()), 

    # Cabeça, olhos e chifres
    Square(x0=0.6, y0=0.6, side=0.2, color=random_color()),
    Circle(x0=0.55, y0=0.65, radius=0.03, color=random_color()), 
    Circle(x0=0.65, y0=0.65, radius=0.03, color=random_color()), 
    Triangles(vertices=np.array([(0.4, 0.7), (0.5, 0.7), (0.5, 0.8)]), color=random_color()), 
    Triangles(vertices=np.array([(0.7, 0.7), (0.8, 0.7), (0.7, 0.8)]), color=random_color()), 
])

session.add_components((rectangles, triangles, circles, mimosa))

# Eventos de teclado e de mouse

In [3]:
# Closure para transformação de translação
def translate(session):
    tx = 0.0
    ty = 0.0
    def closure(action):
        nonlocal tx, ty, session
        tx += 0.01
        ty += 0.01
        session.set_transformation(translate3D(tx, ty))
    return closure

# Closure para transformação de escala
def scale(session):
    sx = 1.0
    sy = 1.0
    def closure(action):
        nonlocal sx, sy, session
        sx -= 0.01
        sy -= 0.01
        session.set_transformation(scale3D(sx, sy))
    return closure

# Closure para transformação de rotação
def rotate(session):
    t = 0.0
    def closure(action):
        nonlocal t, session
        t += np.pi/8
        session.set_transformation(zrotate3D(t))
    return closure

# Transformação das cores
def change_color(action):
    for cp in session.components:
        if not isinstance(cp, OpenGLComponentCompound):
            cp.color = np.hstack([np.random.uniform(0.0, 1.0, size=3), (1.0)])

# Limpeza de transformações
def clear_transformation(action):
    session.clear_transformation()

# Transformação da mimosa
def transform_mimosa(mimosa):

    # Parâmetros
    tx = 0.0
    ty = 0.0
    sx = 1.0
    sy = 1.0
    t = 0.0
    def closure(action):

        # Atualização dos parâmetros
        nonlocal tx, ty, sx, sy, t, mimosa
        tx += 0.001
        ty += 0.001
        sx -= 0.01
        sy -= 0.01
        t += np.pi/8

        # Matrizes
        trans = translate3D(tx, ty)
        scale = scale3D(sx, sy)
        rotat = relative_zrotate3D(t, x0=0.35, y0=0.4)

        # Transformação
        mat = np.dot(trans, scale)
        mat = np.dot(mat, rotat)
        mimosa.transformation = mat
        
    return closure

# Atribuição dos eventos de tecla
window.add_key_callback("RIGHT", translate(session))
window.add_key_callback("DOWN", scale(session))
window.add_key_callback("C", change_color)
window.add_key_callback("DELETE", clear_transformation)
window.add_key_callback("M", transform_mimosa(mimosa))
window.add_mouse_callback(0, rotate(session))

# Execução

In [4]:
# Processamento dos buffers
session.process_buffers()

# Programa principal
for _ in window.display():

    # Limpeza da janela
    session.clear()

    # Renderização
    session.render()